# Демонстрация IntellectShield

Эта тетрадь демонстрирует основные возможности библиотеки IntellectShield для обнаружения аномалий.

In [ ]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Настройка отображения графиков
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

# Для воспроизводимости результатов
np.random.seed(42)

## 1. Генерация синтетических данных

Сначала создадим набор синтетических данных с аномалиями для демонстрации.

In [ ]:
# Генерация нормальных данных
n_samples = 1000
n_outliers = 50

# Создаем два кластера нормальных данных
X1 = np.random.multivariate_normal([0, 0], [[1, 0.5], [0.5, 1]], n_samples // 2)
X2 = np.random.multivariate_normal([5, 5], [[1, -0.5], [-0.5, 1]], n_samples // 2)
X_normal = np.vstack([X1, X2])

# Генерация выбросов равномерно распределенных в пространстве признаков
X_outliers = np.random.uniform(-2, 8, (n_outliers, 2))

# Объединение нормальных данных и выбросов
X = np.vstack([X_normal, X_outliers])

# Создаем метки: 1 для нормальных, -1 для аномалий
y = np.ones(n_samples + n_outliers)
y[n_samples:] = -1

# Визуализация данных
plt.figure(figsize=(10, 8))
plt.scatter(X[:n_samples, 0], X[:n_samples, 1], c='blue', alpha=0.5, label='Нормальные данные')
plt.scatter(X[n_samples:, 0], X[n_samples:, 1], c='red', alpha=0.8, label='Аномалии')
plt.title('Синтетический набор данных с аномалиями')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.legend()
plt.grid(True)
plt.show()

## 2. Использование детектора Isolation Forest

In [ ]:
# Импорт детектора из библиотеки IntellectShield
from intellectshield.detectors.isolation_forest import IsolationForestDetector

# Создаем и обучаем детектор
iso_forest = IsolationForestDetector(n_estimators=100, contamination=0.05)
iso_forest.fit(X)

# Получаем предсказания
y_pred_iso = iso_forest.predict(X)

# Получаем оценки аномальности
scores_iso = iso_forest.score_samples(X)

# Визуализация результатов
plt.figure(figsize=(12, 10))

plt.subplot(2, 1, 1)
plt.scatter(X[:, 0], X[:, 1], c=y_pred_iso, cmap='coolwarm', alpha=0.7)
plt.colorbar(label='Предсказание (-1: аномалия, 1: норма)')
plt.title('Результаты обнаружения аномалий с помощью Isolation Forest')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')

plt.subplot(2, 1, 2)
plt.scatter(X[:, 0], X[:, 1], c=scores_iso, cmap='viridis', alpha=0.7)
plt.colorbar(label='Оценка аномальности')
plt.title('Оценки аномальности (более низкие значения соответствуют аномалиям)')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')

plt.tight_layout()
plt.show()

## 3. Использование детектора LOF (Local Outlier Factor)

In [ ]:
from intellectshield.detectors.lof import LOFDetector

# Создаем и обучаем LOF детектор
lof = LOFDetector(n_neighbors=20, contamination=0.05)
lof.fit(X)

# Получаем предсказания
y_pred_lof = lof.predict(X)

# Визуализация результатов
plt.figure(figsize=(10, 8))
plt.scatter(X[:, 0], X[:, 1], c=y_pred_lof, cmap='coolwarm', alpha=0.7)
plt.colorbar(label='Предсказание (-1: аномалия, 1: норма)')
plt.title('Результаты обнаружения аномалий с помощью LOF')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.show()

## 4. Использование ансамблевого детектора

In [ ]:
from intellectshield.detectors.ensemble import EnsembleAnomalyDetector

# Создаем базовые детекторы
iso_forest_base = IsolationForestDetector(n_estimators=100, contamination=0.05)
lof_base = LOFDetector(n_neighbors=20, contamination=0.05)

# Создаем ансамблевый детектор
ensemble = EnsembleAnomalyDetector(
    detectors=[iso_forest_base, lof_base],
    voting='majority'
)

# Обучаем ансамбль
ensemble.fit(X)

# Получаем предсказания
y_pred_ensemble = ensemble.predict(X)

# Визуализация результатов
plt.figure(figsize=(10, 8))
plt.scatter(X[:, 0], X[:, 1], c=y_pred_ensemble, cmap='coolwarm', alpha=0.7)
plt.colorbar(label='Предсказание (-1: аномалия, 1: норма)')
plt.title('Результаты обнаружения аномалий с помощью ансамблевого детектора')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.show()

## 5. Сравнение детекторов

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Преобразуем метки для работы с метриками sklearn (1=аномалия, 0=нормальное)
y_true_binary = (y == -1).astype(int)
y_pred_iso_binary = (y_pred_iso == -1).astype(int)
y_pred_lof_binary = (y_pred_lof == -1).astype(int)
y_pred_ensemble_binary = (y_pred_ensemble == -1).astype(int)

# Рассчитываем метрики для всех детекторов
metrics = pd.DataFrame({
    'Детектор': ['Isolation Forest', 'LOF', 'Ensemble'],
    'Precision': [
        precision_score(y_true_binary, y_pred_iso_binary),
        precision_score(y_true_binary, y_pred_lof_binary),
        precision_score(y_true_binary, y_pred_ensemble_binary)
    ],
    'Recall': [
        recall_score(y_true_binary, y_pred_iso_binary),
        recall_score(y_true_binary, y_pred_lof_binary),
        recall_score(y_true_binary, y_pred_ensemble_binary)
    ],
    'F1': [
        f1_score(y_true_binary, y_pred_iso_binary),
        f1_score(y_true_binary, y_pred_lof_binary),
        f1_score(y_true_binary, y_pred_ensemble_binary)
    ]
})

print(metrics)

# Визуализация сравнения
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, metric in enumerate(['Precision', 'Recall', 'F1']):
    sns.barplot(x='Детектор', y=metric, data=metrics, ax=axes[i])
    axes[i].set_ylim(0, 1)
    axes[i].set_title(metric)

plt.tight_layout()
plt.show()

## 6. Заключение

В этой демонстрации мы показали использование различных детекторов аномалий из библиотеки IntellectShield:

1. **Isolation Forest** - Эффективный алгоритм, который хорошо работает с высокоразмерными данными
2. **Local Outlier Factor (LOF)** - Алгоритм, основанный на плотности, который хорошо выявляет локальные аномалии
3. **Ансамблевый детектор** - Объединяет преимущества различных алгоритмов для достижения более высокой точности

Каждый детектор имеет свои сильные и слабые стороны, и выбор конкретного алгоритма зависит от характеристик данных и требований задачи.